# Experiments for graph compressioon

In [ ]:
"""parallel implementation of graph coding in universal case considering d_max for common neighbors"""
import numpy as np
import networkx as nx
import GCA_dmax
import multiprocessing as mp
from timeit import default_timer as timer
import pickle

# @jit(nopython=True)
def h(p):
    if type(p) is not list:
        p = np.array([p,1-p])
    x = -(p * np.log2(p))
    x = [0 if np.isnan(i) else i for i in x]
    return np.sum(x)

def process_graph(n,G):
    """Processes graph. Can be used for parallel processing"""
    pg = GCA_dmax.CS12s()
    pg.parse(n, G, update=True)
    
    ph = 2*G.number_of_edges()/ n**2
    cconst = np.log2(np.pi / 8)
    
    lab_iid = n * (n-1) / 2 * h(ph) + 1/2 * np.log2(n * (n-1) / 2) + cconst # labeled IID codelength
    iid_class1 = pg.codelength_bin() # structure IID (IID class1)
    iid_class2 =  pg.codelength_degree() + n - 0.5 * np.log2(n) # Universal structure degree (IID class2)
    tri_class1 = pg.codelength_class1(ctype='tri') # Triangle class1
    tri_class2 = pg.codelength_class2(ctype='tri') +n - 0.5 * np.log2(n) # Triangle class2
    com_class1 = pg.codelength_class1(ctype='com') # Common neighbors class1
    com_class2 = pg.codelength_class2(ctype='com') + n - 0.5 * np.log2(n) # Common neighbors class2
    node4_class1 = pg.codelength_class1(ctype='4nod') # 4-node class1
    node4_class2 = pg.codelength_class2(ctype='4nod')+ n - 0.5 * np.log2(n) # 4-node class2
    
    print('end of graph:', n)
    return (lab_iid, iid_class1, iid_class2, tri_class1, tri_class2,
           com_class1, com_class2, node4_class1, node4_class2)


class CodeData:
    def __init__(self):
        self.lab_iid = []
        self.iid_class1 = []
        self.iid_class2 = []
        self.tri_class1 = []
        self.tri_class2 = []
        self.com_class1 = []
        self.com_class2 = []
        self.node4_class1 = []
        self.node4_class2 = []

parallel = True
CD = CodeData()
s = timer()
nv = ['Datasets/genetic.txt', 'Datasets/economic.txt', 
      'Datasets/transportation.txt', 'Datasets/wikipedia.txt',
      'Datasets/collaboration.txt', 'Datasets/internet.txt']
if parallel:
    pool = mp.Pool(processes=20)
    results = []
    for i in nv: # this loop can be used to go over all real-world graphs at a time
        G = nx.read_edgelist(i, nodetype=int)
        G = nx.convert_node_labels_to_integers(G)
        n = G.number_of_nodes()
        results.append(pool.apply_async(process_graph, args=(n, G)))
    for p in results:
        result = p.get()
        CD.lab_iid.append(result[0])
        CD.iid_class1.append(result[1])
        CD.iid_class2.append(result[2])
        CD.tri_class1.append(result[3])
        CD.tri_class2.append(result[4])
        CD.com_class1.append(result[5])
        CD.com_class2.append(result[6])
        CD.node4_class1.append(result[7])
        CD.node4_class2.append(result[8])
        
        
else:
    for i in nv:
        G = nx.read_edgelist(i, nodetype=int)
        G = nx.convert_node_labels_to_integers(G)
        n = G.number_of_nodes()
        result = process_graph(n,G)
        CD.lab_iid.append(result[0])
        CD.iid_class1.append(result[1])
        CD.iid_class2.append(result[2])
        CD.tri_class1.append(result[3])
        CD.tri_class2.append(result[4])
        CD.com_class1.append(result[5])
        CD.com_class2.append(result[6])
        CD.node4_class1.append(result[7])
        CD.node4_class2.append(result[8])

e = timer()
print("Total time: ",e-s)
print()
print("Labeled iid: ", CD.lab_iid)
print("IID class1: ", CD.iid_class1)
print("Degree: ", CD.iid_class2)
print("Triangle class1: ", CD.tri_class1)
print("Triangle class2: ", CD.tri_class2)
print("Common neighbors class1: ", CD.com_class1)
print("Common neighbors class2: ", CD.com_class2)
print("4-node class1: ", CD.node4_class1)
print("4-node class2: ", CD.node4_class2)